In [ ]:
from mlxtend.data import loadlocal_mnist
import platform
import matplotlib.pyplot as plt
import numpy as np
if platform.system()=='Windows':
        X,y=loadlocal_mnist(
            images_path='gzip\emnist-mnist-train-images-idx3-ubyte\emnist-mnist-train-images-idx3-ubyte',
            labels_path='gzip\emnist-mnist-train-labels-idx1-ubyte\emnist-mnist-train-labels-idx1-ubyte'
        )
        #X:(60000,784) i.e. 60000images, size:28*28
img=[]
for images in X:
    img.append(np.array(images).reshape(28,28))

plt.imshow(img[59998])
plt.show()
print(y)